In [ ]:
import sklearn
import matplotlib
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import numpy as np
import sys
from numpy.linalg import norm #get vector normalization
from scipy import optimize#OPTIMIZE DAT SHET
import matplotlib.pyplot as plot
%matplotlib inline

amount = input("How fine should the training data be? (higher the number the finer the data): ");
HoSleep = input("How many hours did you sleep?: ");
HoStudy = input("How many hours did you study?: ");

try:
    amount = int(amount);
except:
    amount = 100;

#Supervised regression method, sup = inp & out, reg = predic of num
X = np.array(([3, 5], [5, 1], [10, 2]), dtype=float);#[hrs sleep, hrs study]
y = np.array(([75], [82], [93]), dtype=float);#[score]

print("Initial X",X);
print("Initial Y",y);
print();

#Need to account for unit diff. Inp is in hrs, out is in grade num
#Will scale data
#Data is positive, so will divide by max val of each var
X = X/np.amax(X, axis = 0);#Compare to the highest number available
y = y/100;#We know that highest score possible is 100
#This network must follow the format of two inputs one output
#Perspective because it is looking at everything as compared to the largest sample given, and makes a percent thereof
print("Perspective X",X);
print("Perspective Y",y);
#In between the inp and out is a hidden network, consisting of neurons
"""Things that connect neurons are synapses,
    synapses take input values, and multiply them by a weight before
    passing them on. Neurons also take multiple inputs, from
    all their synapses, and apply an activation function."""
print();

#building NN as a class
class Neural_Network(object):#Make class
    def __init__(self):#Init method
        #use self."variable" to make it class accessible
        #Define HyperParameters
        self.inputLayerDepth = 2;#Two inputs
        self.outputLayerDepth = 1;#One output
        self.hiddenLayerDepth = 3;#Three neurons to calculate with
        self.epsilon = 1e-4;
        
        #Weights (Parameters)
        self.W1 = np.random.randn(self.inputLayerDepth, \
                                self.hiddenLayerDepth);
        self.W2 = np.random.randn(self.hiddenLayerDepth, \
                                 self.outputLayerDepth);
    
    def forward(self, X):#Use matrices to input multiple vals at once
        #Propogate inputs through network
        #Matrix multiplication to speed things up
        #Input matrix = X, weight matrix = w(1), out matrix = Z(2)
        #Z(2) is 3 by 3, one row per example, one col per hidden unit
        #Func: Z(2) = Xw(1)
        #a(2) = f(z(2))
        self.z2 = np.dot(X, self.W1);
        self.a2 = self.sigmoid(self.z2);
        self.z3 = np.dot(self.a2, self.W2);
        yHat = self.sigmoid(self.z3);
        return yHat
    
    def sigmoid(self, z):
        #Apply sigmoid activation function
        return 1/(1+np.exp(-z));
    
    def sigmoidPrime(self, z):#Derivation of the Sigmoid Function
        return np.exp(-z)/((1+np.exp(-z))**2);
    
    def cFunction(self, X, y):
        #Compute cost for given X,y, use weights already stored in class.
        self.yHat = self.forward(X)
        J = 0.5*sum((y-self.yHat)**2)
        return J
    
    def cFPrime(self, X, y):#Get derivs with context of W1 and W2
        self.yHat = self.forward(X);
        
        #Delta of z3 data calc
        delta3 = np.multiply(-(y-self.yHat), self.sigmoidPrime(self.z3));
        dW2 = np.dot(self.a2.T, delta3);
        
        #Delta of z2 data calc, among everything else I think
        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2);
        dW1 = np.dot(X.T, delta2);
        
        return dW1, dW2;
    
    def getParams(self):
        #GET W1 and W2 rolled into a vector:
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()));
        return params;
    
    def setParams(self, params):
        #SET W1 and W2 using a single parameter vector:
        W1_start = 0;
        W1_end = self.hiddenLayerDepth*self.inputLayerDepth;
        self.W1 = np.reshape(params[W1_start:W1_end], \
                            (self.inputLayerDepth, self.hiddenLayerDepth));
        W2_end = W1_end+self.hiddenLayerDepth*self.outputLayerDepth;
        self.W2 = np.reshape(params[W1_end:W2_end], \
                            (self.hiddenLayerDepth, self.outputLayerDepth));
        
    def calcGradients(self, X, y):
        dW1, dW2 = self.cFPrime(X, y);
        return np.concatenate((dW1.ravel(), dW2.ravel()));
    
    ### EXPERIMENTAL CODE ###
    def gradNormalizer1(self, x):
        return x**2;
    
    def gradNormalizer2(self, N, X, y):
        paramsInitial = N.getParams();
        numgrad = np.zeros(paramsInitial.shape);
        perturb = np.zeros(paramsInitial.shape);
        e = 1e-4;
        
        for a in range(len(paramsInitial)):
            #Setting pert vector
            perturb[a] = e;
            N.setParams(paramsInitial+perturb);
            loss2 = N.cFunction(X, y);
            
            N.setParams(paramsInitial-perturb);
            loss1 = N.cFunction(X, y);
            
            #Compute slope between values:
            numgrad[a] = (loss2-loss1)/(2*e);
            
            #Return to zero
            perturb[a] = 0;
            
        #Reset parameters:
        N.setParams(paramsInitial);
        return numgrad;
    
    def gradNormalizer3(self, val):
        numericGradient = (self.gradNormalizer1(val+self.epsilon)-self.gradNormalizer1(val-self.epsilon))/(2*self.epsilon);
        return numericGradient;
        

NN = Neural_Network();#Construct NN shortcut
yHat = NN.forward(X);#Parse yHat data
cost1 = NN.cFunction(X, y);#Parse cost step one
dW1, dW2 = NN.cFPrime(X, y);#Zip Values of delta calcs
print("yHat (rough predict of change of values):", yHat);#Let's see what this crap is
print("Y (rough predict)",y);
print();#Organization of prints
print("delta W1:", dW1);#Derivitive of J in respect to derivitive of W (Find the value of the slope at all values quickly) for 1 and 2
print("delta W2:", dW2);

scalar = 3;#Scale the calculation gradient (I believe)
NN.W1 = NN.W1+scalar+dW1;
NN.W2 = NN.W2+scalar+dW2;
cost2 = NN.cFunction(X, y);#Cost val 2

print();#Organization of prints
print("Cost 1&2:",cost1, cost2);#Print it

dW1, dW2 = NN.cFPrime(X, y);#Redoe delta calcs in zip
NN.W1 = NN.W1-scalar+dW1;
NN.W2 = NN.W2-scalar+dW2;
cost3 = NN.cFunction(X, y);

print();#Organization of prints
print("Cost 2&3:",cost2, cost3);#Print it
print();

class trainer(object):
    def __init__(self, N):
        #Scope out to see NN
        self.N = N;
        
    def cFWrapper(self, params, X, y):#Wrap values
        self.N.setParams(params);
        cost = self.N.cFunction(X, y);
        grad = self.N.calcGradients(X, y);
        return cost, grad;
    
    def cbF(self, params):
        self.N.setParams(params);
        self.J.append(self.N.cFunction(self.X, self.y));
        
    
    def train(self, X, y):
        
        #Making internal variable for callback func
        self.X = X;
        self.y = y;
        #Now store the costs:
        self.J = [];
        
        params0 = self.N.getParams();
        
        options = {'maxiter': 200, 'disp': True};
        _res = optimize.minimize(self.cFWrapper, params0, \
                                 jac = True, method='BFGS', \
                                 args = (X, y), options = options, \
                                 callback = self.cbF);
        #Replace random parameters with trained parameters
        self.N.setParams(_res.x);
        self.optResults = _res;

graph1 = plot.figure(1);
tValues = np.arange(-5, 5, 0.01);
plot.plot(tValues, NN.sigmoid(tValues), linewidth = 2);
plot.plot(tValues, NN.sigmoidPrime(tValues), linewidth = 2);
plot.grid(1);
plot.legend(['sigmoid', 'sigmoidPrime']);
T = trainer(NN);
T.train(X, y);
graph2 = plot.figure(2);
plot.plot(T.J, linewidth = 2);
plot.grid(1);
plot.ylabel('Cost');
plot.xlabel('Iterations');
#Test network for combos
hoursSleep = np.linspace(0, 10, amount);
hoursStudy = np.linspace(0, 5, amount);
#print(hoursSleep, "\n\n\n", hoursStudy);
#print();
#Normalizing data:
hoursSleepNorm = hoursSleep/10.;
hoursStudyNorm = hoursStudy/5.

#Flatten Data:
a, b = np.meshgrid(hoursSleepNorm, hoursStudyNorm);

#Put it in the Matrix
allInputs = np.zeros((a.size, 2));
allInputs[:, 0] = a.ravel();
allInputs[:, 1] = b.ravel();
allOut = NN.forward(allInputs);
print(allOut);
print();
print();
#print(NN.forward(X));

graph3 = plot.figure(3);
yy = np.dot(hoursStudy.reshape(amount, 1), np.ones((1, amount)));
xx = np.dot(hoursSleep.reshape(amount, 1), np.ones((1, amount))).T;

CS = plot.contour(xx, yy, 100*allOut.reshape(amount, amount));
plot.clabel(CS, inline = 1, fontsize = 10);
plot.xlabel("Hours Slept");
plot.ylabel("Hours Studied");